In [1]:
covariance_matrix = [[0.2, 0.4, -0.5],
                     [-0.3, 0.7, 1.0],
                     [0.9, -0.45, 0.5]]

n = len(covariance_matrix)


In [2]:
# Task 1
# LU decomposition using Gauss Elimination


def generate_null_matrix(dimension):
    elementary_matrix = [[] for i in range(dimension)]
    for i in range(dimension):
        for j in range(dimension): 
            elementary_matrix[i].append(0)
    return elementary_matrix


def square_matrix_multiplication(matrix_a, matrix_b):
    result_matrix = generate_null_matrix(len(matrix_a))
    for i in range(n):
        for j in range(n):
            for k in range(n):
                result_matrix[i][j] += matrix_a[i][k] * matrix_b[k][j]
    return result_matrix


def ones_column(diagonal_element, elementary_matrices_array, temporary_matrix):
    el = generate_null_matrix(len(temporary_matrix))
    for i in range(n):
        if(i < diagonal_element):
            el[i][i] = 1.0 
        else:
            if(temporary_matrix[i][diagonal_element] == 0.0):
                el[i][i] = 1.0 
            else:
                el[i][i] = 1.0 / temporary_matrix[i][diagonal_element]
    elementary_matrices_array.append(el)
    temporary_matrix = square_matrix_multiplication(el, temporary_matrix)
    return elementary_matrices_array, temporary_matrix

def zeroes_column(diagonal_element, elementary_matrices_array, temporary_matrix):
    el = generate_null_matrix(len(temporary_matrix))
    for i in range(n):
        el[i][i] = 1.0
        if(i > diagonal_element):
            if(temporary_matrix[i][diagonal_element] == 0):
                el[i][diagonal_element] = 0.0
            else:
                el[i][diagonal_element] = -1.0
    elementary_matrices_array.append(el)
    temporary_matrix = square_matrix_multiplication(el, temporary_matrix)
    return elementary_matrices_array, temporary_matrix

def cut_i_row_and_jth_column(matrix, row, column):
    temp = []
    for i in range(len(matrix)):
        if(i != row):
            temp.append(matrix[i].copy())
    for i in range(len(temp)):
        temp[i].pop(column)
    return temp

def determinant(matrix):
    if(len(matrix) == 1):
        return matrix[0][0]
    det = 0
    for i in range(len(matrix)):
        if i % 2 == 0:
            det += matrix[0][i] * determinant(cut_i_row_and_jth_column(matrix, 0, i))
        else:
            det -= matrix[0][i] * determinant(cut_i_row_and_jth_column(matrix, 0, i))
    return det

def transpose(matrix):
    for i in range(len(matrix) - 1 ):
        for j in range(i + 1, len(matrix)):
            matrix[i][j], matrix[j][i] = matrix[j][i], matrix[i][j]
    return matrix

def inverse(matrix):
    inverse = generate_null_matrix(len(matrix))
    det = determinant(matrix)
    if det == 0:
        print("non-invertible")
        return inverse
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if (i + j) % 2 == 0: 
                inverse[i][j] = determinant(cut_i_row_and_jth_column(matrix, i, j)) / det
            else:
                inverse[i][j] = -determinant(cut_i_row_and_jth_column(matrix, i, j)) / det
    return transpose(inverse)

def round_matrix(matrix):
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            matrix[i][j] = round(matrix[i][j], 2)
    return matrix


In [3]:
elementary_matrices_array = []

Upper_triangular = covariance_matrix

for i in range(n):
    elementary_matrices_array, Upper_triangular = ones_column(i, elementary_matrices_array, Upper_triangular)
    elementary_matrices_array, Upper_triangular = zeroes_column(i, elementary_matrices_array, Upper_triangular)

Lower_triangular = elementary_matrices_array[-1]

for i in range(len(elementary_matrices_array) - 2, -1, -1):
    Lower_triangular = square_matrix_multiplication(Lower_triangular, elementary_matrices_array[i])
Lower_triangular = inverse(Lower_triangular)

In [4]:
print("***Gauss LU decomposition***")
print("\nInitial covariance matrix:")
print(covariance_matrix)
print("\nL matrix:")
print(round_matrix(Lower_triangular))
print("\nU matrix:")
print(round_matrix(Upper_triangular))
print("\nMultiplied:")
print(round_matrix(square_matrix_multiplication(Lower_triangular, Upper_triangular)))

***Gauss LU decomposition***

Initial covariance matrix:
[[0.2, 0.4, -0.5], [-0.3, 0.7, 1.0], [0.9, -0.45, 0.5]]

L matrix:
[[0.2, -0.0, 0.0], [-0.3, 1.3, -0.0], [0.9, -2.25, 3.18]]

U matrix:
[[1.0, 2.0, -2.5], [0.0, 1.0, 0.19], [0.0, 0.0, 1.0]]

Multiplied:
[[0.2, 0.4, -0.5], [-0.3, 0.7, 1.0], [0.9, -0.45, 0.5]]


In [5]:
#Task 2
# A = QR decomposition (or QU), Q - orhogonal, R - Upper triangular
#The QR decompostion always exists, even if the matrix does not have full rank
#If A is nonsingular, then this factorization is unique.
#https://www.math.ucla.edu/~yanovsky/Teaching/Math151B/handouts/GramSchmidt.pdf
#used in OLS and to calculate eigenvalues (Ax=lambda*x trick does not work on computer)
#Orthogonal matrix, or orthonormal matrix, is a real square matrix whose columns and rows are orthonormal vectors.
#Orthogonal means Q*Q^t = Q^t*Q = I and Q^t=Q^(-1)
#A set of vectors form an orthonormal set if all vectors in the set are mutually orthogonal and all of unit length
#The algotihmuses Gram-Schmidt process

#All symmetric matrices (with real number entries) have a full set of eigenvalues and eigenvectors.
#The eigenvalues are all real numbers.
#The eigenvectors corresponding to different eigenvalues are orthogonal 
#(eigenvectors of different eigenvalues are always linearly independent, 
#the symmetry of the matrix buys us orthogonality).



